In [ ]:
#Connect to drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/feature_selection/heart"

/content/drive/MyDrive/feature_selection/heart


In [ ]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.3 MB 4.3 MB/s 
     |████████████████████████████████| 409 kB 55.9 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.


In [ ]:
#Import library
import ortools
import numpy as np
from ortools.linear_solver import pywraplp
import pandas as pd
import math
from pyitlib import discrete_random_variable as drv

In [ ]:
#Initialize solver
solver=pywraplp.Solver.CreateSolver('SCIP')

STEP 1: READ DATA AND CREATE FEATURES, LABEL




In [ ]:
#Read data from csv file
df = pd.read_csv("heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [ ]:
#Feature and label
y = df['target'].values

X = df.drop(labels='target', axis=1).values

STEP 2: CALCULATE PRE MUTUAL INFORMATION

In [ ]:
#Mutual information between features and label
mutual_info = []
for i in range(X.shape[1]):
    temp = drv.information_mutual(X[:, i], y)
    mutual_info.append(temp)

In [ ]:
#Mutual information between features
cost = np.zeros(shape=(X.shape[1], X.shape[1]), dtype='float32')
for i in range(X.shape[1]):
    for j in range(X.shape[1]):
        cost[i][j] = drv.information_mutual(X[:, i], X[:, j])

In [ ]:
#mutual information conditional
conditional = np.zeros(shape=(X.shape[1], X.shape[1]), dtype='float32')
for i in range(X.shape[1]):
    for j in range(X.shape[1]):
      conditional[i][j] = drv.information_mutual_conditional(X[:, j], X[:, i], y)

print(conditional)

STEP 3: TAKE MUTUAL INFORMATION FROM PRE-CALCULATED FILES

In [ ]:
#Read data from file csv
label = pd.read_csv("label.csv")
labl = label.to_numpy()
mutual_info = labl.reshape(-1)
cost1 = pd.read_csv("mutual.csv")
cost = cost1.to_numpy()
cond = pd.read_csv("mutual_con.csv")
conditional = cond.to_numpy()
print(conditional.shape)

(13, 13)


STEP 4: MODEL INTEGER PROGRAMMING

In [ ]:
#tạo các biến
print("Starting...")
x = {}
for i in range(X.shape[1]):
    for j in range(X.shape[1]):
        if (j > i):
            x[i, j] = solver.IntVar(0, 1, '')
d =[0]*X.shape[1]

for i in range(X.shape[1]):
    d[i] = solver.IntVar(0, 1, '')
# solver.Add(solver.Sum([d[i] for i in range(X.shape[1])]) == 5)

#add constraint
print("running....")
for i in range(X.shape[1]):
    for j in range(X.shape[1]):
        if(j > i):
            # solver.Add((d[i]+d[j])-1<=2*x[i, j])
            # solver.Add(2*x[i, j]<=d[i]+d[j])
            solver.Add(x[i, j] <= d[i])
            solver.Add(x[i, j] <= d[j])
            solver.Add(x[i, j] >= d[i] + d[j] - 1)

objective_term = []

print("objective....")
for i in range(X.shape[1]):

    Ik = (1-d[i])* mutual_info[i]


    cij = 0
    for j in range(X.shape[1]):
        if (j > i):
            cij += d[j]*(-cost[i, j]+conditional[i][j]) - x[i, j]*(-cost[i, j]+conditional[i][j])
        elif(i > j):
            cij += d[j]*(-cost[i, j]+conditional[i][j]) - x[j, i]*(-cost[i, j]+conditional[i][j])

    objective_term.append(Ik+cij+d[i])

solver.Minimize(solver.Sum(objective_term))
status = solver.Solve()
res = []
print("Results...")
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n')
    for i in range(X.shape[1]):
        if d[i].solution_value() > 0.5:
            print(f'Feauture {i} is selected')
            res.append(1)
        else:
            res.append(0)

    for i in range(X.shape[1]):
        for j in range(X.shape[1]):
            if j > i and x[i,j].solution_value()> 0.5:
                print("features ", i , ", " , j , " are selected")

    print('Time = ', solver.WallTime(), ' milliseconds')
    print('Problem solved in %d iterations' % solver.iterations())
    print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

solver.Clear()